[https://pypi.org/project/transformers/](https://pypi.org/project/transformers/)

[Attention is all you need](https://arxiv.org/pdf/1706.03762)


## Transformer revolutionen

### Attention

En _attention_ mekanism förklaras ofta med en databas-lik analogi där $q$ är en _query_, $k$ är en _key_ och $v$ är ett värde. Intuitivt kan vi tänka på det som att $q$ och $k\in\mathcal{D}$ letar upp värden i en kontext, till och med en linjärkombination av _väntevärden_ i ett urval av inlärda mönster, som avgör inte bara om $v$ från indatan är relevant men också på vilket sätt genom att de ligger nära liknande koncept i den latenta rymden (därav namnet 'attention', $\alpha$ nedan är en funktion kallad en _attention mechanism_).

$\textrm{Attention}(\mathbf{q}, \mathcal{D}) \stackrel{\textrm{def}}{=} \sum_{i=1}^m \alpha(\mathbf{q}, \mathbf{k}_i) \mathbf{v}_i$



Högerledet i ovan är slående likt optimeringsvillkoret för SVM. Vi kan tolka det som en projektion av indatan till något högdimensionellt rum i sin kontext (hela indatan + inbäddningar), där vi sedan använder nätverkets _inlärda_ $\alpha$ vikter för att kollapsa ner dimensionaliteten igen. På så sätt är Attention är helt enkelt en sorts konvolution i det matematiska avseendet. Den ursprungliga Transformer-arkitekturen fungerar enligt följande:

1. Källtexten behandlas av en _encoder_ som producerar en inbäddning per token (en konvolution från en sekvens av tecken till en sekvens av inbäddningar)
2. Inbäddningarna matas sedan till en _decoder_ tillsammans med sekvensen av hittils producerad utdata från avkodaren. (en till konvolution)
3. Avkodaren förutsäger nästa token för varje token+inbäddning i indatan (ytterligare en konvolution, varje förutsägelse är en token i 'utlagret').
4. Den _sista_ token genererad i steg 3 läggs till utdatan. (konvolution igen)
5. 2-4 upprepas tills en end-of-sequence token genereras. (rekursion)

Vidare utveckling visade att det oftast räcker med endera _encoder_ eller _decoder_ -- vilket leder till mycket bättre prestanda. Men vissa problem tjänas fortfarande av encoder-decoder modeller, särskilt översättning och sammanfattning.

'Encoder only' - BERT

'Decoder only' - GPT


Med insikten att attention är en algoritmisk 1D konvolution (ett 'kernel-trick') som blandar beräkningsmodeller (NN och 'vanlig' beräkning) är det inte förvånande att arkitekturen fungerar även för bildbehandling och policynätverk i RL. Men det tog många med överraskning -- till exempel författaren till kurslitteraturen. Exempel som detta är en stor anledning att jag är så kritisk mot antropomorfiska termer som 'attention' eller 'reasoning' när det gäller AI. Notera att Attention genererar en linjärkombination av termer i $\mathcal{D}$. Ett namn som 'Support Query Convolution' eller 'Adaptive Context Search' vore mycket mer beskrivande (enligt mig förstås). 

Exakt vad $q,k$ och $v$ kommer ifrån beror på vilken sorts arkitektur det rör sig om. I en encoder-decoder arkitekturen kommer K och V från encodern medans Q är i decodern. I encoder- och decoder-only kommer alla från samma källa (så kallad self-attention). 

$Q, K, V$ är matriser med alla värdena $q,k,v$. De defineras så här:
\begin{align*}Q&=HW^Q \\
K&=HW^K \\
V&=HW^V\end{align*}

Där $H$ från början är alla inbäddningarna (tokens och positioner) i indatan och $W$ matriserna har inlärda vikter. I varje lager blir sedan $H$ utdatan från förra lagret. Återigen en matematisk konvolution!

### Multi-head attention

Precis som SVM kernels finns det flera olika attention-mekanismer. För att fortsätta databas- och lagringsanalogin kan vi tänka oss att vi har flera 'läshuvuden' som samtidigt processar en sekvens. Detta motsvarar att vi kör många kernels i en CNN över ett lager. Attention-mekanismen (kernel funktionen) kallas _scaled dot-product attention_:

$\textrm{Attention}(\mathbf{Q}, \mathbf{K}, \mathbf{V}) = \textrm{softmax}(\frac{\mathbf{Q}\mathbf{K}^T}{\sqrt{d_K}})\mathbf{V}$


$QK^T$ innehåller en _similarity score_, dvs en skalärprodukt, för varje par $(q,k) \in Q×K$. Detta kallas det _kvadratiska kontextfönstret_. 

<img src="../Data/multihead.png">

### Encoder Only - BERT

Två träningsprocedurer förslås för BERT modellen:

##### MLM
>Masked language model. Varje token i en mening i träningsdatan har en viss sannolikhet att ersättas med en \[MASK\] token. Modellen tränas för att komplettera texten.
##### NSP
>Next sentence prediction. Modellen tränas för att avgöra om en mening följer en annan.

BERTs förtränade modeller tränades simultant med båda dessa procedurer. Se s 597 i boken för detaljer.

Med transformerbiblioteket kan vi relativt enkelt träna egna modeller, men det är mycket resurskrävade.

In [2]:
from transformers import BertConfig, BertForMaskedLM, BertTokenizerFast

bert_tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
config = BertConfig(vocab_size=bert_tokenizer.vocab_size, hidden_size=128, num_hidden_layers=2, num_attention_heads=4, intermediate_size=512, max_position_embeddings=128)
bert = BertForMaskedLM(config)



In [3]:
from datasets import load_dataset
def tokenizer(example, tokenizer=bert_tokenizer):
    return tokenizer(example["text"], truncation=True, max_length=128, padding="max_length")

mlm_dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
mlm_dataset = mlm_dataset.map(tokenizer, batched=True)

In [4]:
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling

args = TrainingArguments(output_dir="../local/my_bert", num_train_epochs=5,per_device_train_batch_size=32)
mlm_collator = DataCollatorForLanguageModeling(bert_tokenizer,mlm=True,mlm_probability=0.15)
trainer = Trainer(model=bert, args=args, train_dataset=mlm_dataset,data_collator=mlm_collator)

In [ ]:
#trainter_output = trainer.train() # ~ tre minuter träning använder ca 2 GB VRAM

Step,Training Loss
500,8.874100
1000,7.452100
1500,7.226900
2000,7.158300
2500,7.100600
3000,7.091000
3500,7.063000
4000,7.059300
4500,7.055700
5000,7.035000


In [5]:
bert = BertForMaskedLM(config).from_pretrained("../local/my_bert/checkpoint-5500")

In [9]:
from transformers import pipeline
fill_mask = pipeline("fill-mask", model=bert, tokenizer=bert_tokenizer)
top_predictions = fill_mask("The capital of [MASK] is Rome.")
top_predictions[0]

Device set to use cuda:0


{'score': 0.07928488403558731,
 'token': 2003,
 'token_str': 'is',
 'sequence': 'the capital of is is rome.'}

In [6]:
args = TrainingArguments(output_dir="../local/my_bert", num_train_epochs=2000,per_device_train_batch_size=256)
mlm_collator = DataCollatorForLanguageModeling(bert_tokenizer,mlm=True,mlm_probability=0.15)
trainer = Trainer(model=bert, args=args, train_dataset=mlm_dataset,data_collator=mlm_collator)

In [ ]:
#trainer_output = trainer.train() # ~ 1 timma träning, 16 GB VRAM

Step,Training Loss
500,7.000300
1000,6.935800
1500,6.874400
2000,6.832300
2500,6.796900
3000,6.763200
3500,6.733400
4000,6.705300
4500,6.682400
5000,6.650300


In [7]:
bert = BertForMaskedLM(config).from_pretrained("../local/my_bert/checkpoint-14400")

In [10]:
fill_mask = pipeline("fill-mask", model=bert, tokenizer=bert_tokenizer)
top_predictions = fill_mask("The capital of [MASK] is Rome.")
top_predictions[0]

Device set to use cuda:0


{'score': 0.07928488403558731,
 'token': 2003,
 'token_str': 'is',
 'sequence': 'the capital of is is rome.'}

In [ ]:
#trainer_output = trainer.train("../local/my_bert/checkpoint-14400") # 8 timmar till träning

There were missing keys in the checkpoint model loaded: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias'].


Step,Training Loss
14500,6.464000
15000,6.448900
15500,6.423700
16000,6.405000
16500,6.382500
17000,6.365200
17500,6.351100
18000,6.335200
18500,6.314600
19000,6.293700


In [11]:
bert = BertForMaskedLM(config).from_pretrained("../local/my_bert/checkpoint-144000")

In [12]:
fill_mask = pipeline("fill-mask", model=bert, tokenizer=bert_tokenizer)
top_predictions = fill_mask("The capital of [MASK] is Rome.")
top_predictions[0]

Device set to use cuda:0


{'score': 0.09136169403791428,
 'token': 16590,
 'token_str': 'skye',
 'sequence': 'the capital of skye is rome.'}

In [ ]:
#trainer_output = trainer.train("../local/my_bert/checkpoint-144000") # 9 timmar till träning

There were missing keys in the checkpoint model loaded: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias'].


Step,Training Loss
144500,4.115600
145000,4.105300
145500,4.105300
146000,4.096700
146500,4.088700
147000,4.076400
147500,4.085300
148000,4.075300
148500,4.069400
149000,4.066600


In [13]:
bert = BertForMaskedLM(config).from_pretrained("../local/my_bert/checkpoint-288000")

In [14]:
fill_mask = pipeline("fill-mask", model=bert, tokenizer=bert_tokenizer)
top_predictions = fill_mask("The capital of [MASK] is Rome.")
top_predictions[0]

Device set to use cuda:0


{'score': 0.0712178573012352,
 'token': 3163,
 'token_str': 'ireland',
 'sequence': 'the capital of ireland is rome.'}